# 1. 데이터 수집 및 전처리

## 1.1 종가/기술지표 데이터

In [3]:
from re import M
import pandas as pd

momentum_data_url = 'https://raw.githubusercontent.com/prometheus-11team/prometheus-11team/main/store%20data/M7_stock_momentum_data_fixed.csv'
momentum_data = pd.read_csv(momentum_data_url)
momentum_data = momentum_data[['Date', 'Ticker', 'Close', 'ROC_10', 'RSI_14', 'MACD']]
momentum_data['Date'] = pd.to_datetime(momentum_data['Date'])
momentum_data = momentum_data[momentum_data['Date'] >= '2020-01-01']
momentum_data = momentum_data.reset_index(drop=True)
print(len(momentum_data))
display(momentum_data.tail())
data_A = momentum_data.copy()

9135


,Date,Ticker,Close,ROC_10,RSI_14,MACD
9130,2025-03-06,TSLA,263.450012,-25.663088,26.904166,-29.827217
9131,2025-03-07,TSLA,262.670013,-22.240964,26.760181,-30.524241
9132,2025-03-10,TSLA,222.149994,-32.789763,20.594190,-33.954855
9133,2025-03-11,TSLA,230.580002,-23.850723,24.492249,-35.583228
9134,2025-03-12,TSLA,248.089996,-14.687068,31.963305,-35.056703


## 1.2 거시경제 지표 데이터

In [ ]:
Macro_data_url = 'https://raw.githubusercontent.com/prometheus-11team/prometheus-11team/main/store%20data/macro_indicators_2020_2025-03.csv'
Macro_data = pd.read_csv(Macro_data_url)
Macro_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
Macro_data['Date'] = pd.to_datetime(Macro_data['Date'])
print(len(Macro_data))
Macro_data.tail()

63


,Date,Federal Funds Rate,10Y Treasury Yield,CPI,Core CPI,PCE Price Index,Retail Sales,Unemployment Rate,Non-Farm Payrolls,M2 Money Stock
58,2024-11-01,4.64,4.36,316.449,322.619,124.399,712145.0,4.2,158619.0,21454.2
59,2024-12-01,4.48,4.39,317.603,323.296,124.769,717662.0,4.1,158942.0,21489.9
60,2025-01-01,4.33,4.63,319.086,324.739,125.189,711461.0,4.0,159053.0,21577.0
61,2025-02-01,4.33,4.45,319.775,325.475,125.600,711680.0,4.1,159170.0,21670.5
62,2025-03-01,4.33,4.28,319.615,325.659,NaN,722025.0,4.2,159398.0,21762.5


###1.2-1 월간 데이터를 일간데이터로

In [ ]:
# 1. 'Date'를 인덱스로 설정
Macro_data.set_index('Date', inplace=True)

# 2. 일단위 date range 생성 (시작일 ~ 종료일 설정 필요)
daily_index = pd.date_range(start='2020-01-01', end='2025-04-30', freq='D')

# 3. 일간 인덱스로 재구성 + 최근 값으로 채우기
Macro_data_daily = Macro_data.reindex(daily_index, method='ffill')

# 4. 인덱스 다시 'Date' 컬럼으로 복원
Macro_data_daily = Macro_data_daily.reset_index().rename(columns={'index': 'Date'})

# 확인
print(len(Macro_data_daily))
Macro_data_daily.head()

1947


,Date,Federal Funds Rate,10Y Treasury Yield,CPI,Core CPI,PCE Price Index,Retail Sales,Unemployment Rate,Non-Farm Payrolls,M2 Money Stock
0,2020-01-01,1.55,1.76,259.127,266.716,104.419,515119.0,3.6,152031.0,15416.8
1,2020-01-02,1.55,1.76,259.127,266.716,104.419,515119.0,3.6,152031.0,15416.8
2,2020-01-03,1.55,1.76,259.127,266.716,104.419,515119.0,3.6,152031.0,15416.8
3,2020-01-04,1.55,1.76,259.127,266.716,104.419,515119.0,3.6,152031.0,15416.8
4,2020-01-05,1.55,1.76,259.127,266.716,104.419,515119.0,3.6,152031.0,15416.8


In [ ]:
# 주가/기술지표 + 거시경제지표
mom_macro_merged = pd.merge(momentum_data, Macro_data_daily, on='Date', how='left')
print(len(mom_macro_merged))
mom_macro_merged.head()
data_C = mom_macro_merged.copy()

9135


## 1.3 기업실적 데이터

In [ ]:
financial_data_url = 'https://raw.githubusercontent.com/prometheus-11team/prometheus-11team/main/store%20data/M7_financial_data_2020_2025.csv'
financial_data = pd.read_csv(financial_data_url)
financial_data.head()

,Company,Fiscal Year,Period,Report Type,End Date,Operating Income,Net Income,EPS Diluted,Total Assets,Shareholders Equity
0,Alphabet,2020,Annual,10-K,2020-12-31,41224.0,40269.0,58.61,319616.0,222544.0
1,Alphabet,2020,Q1,10-Q,2020-03-31,7977.0,6836.0,9.87,273403.0,203659.0
2,Alphabet,2020,Q2,10-Q,2020-06-30,14360.0,13795.0,20.00,278492.0,207322.0
3,Alphabet,2020,Q3,10-Q,2020-09-30,25573.0,25042.0,36.38,299243.0,212920.0
4,Alphabet,2021,Annual,10-K,2021-12-31,78714.0,76033.0,112.20,359268.0,251635.0


### 1.3-1 월간 데이터를 일간데이터로

In [ ]:
# 회사명과 Ticker 간 매핑 딕셔너리
company_to_ticker = {
    'Alphabet': 'GOOGL',
    'Amazon': 'AMZN',
    'Apple': 'AAPL',
    'Meta': 'META',
    'Microsoft': 'MSFT',
    'Nvidia': 'NVDA',
    'Tesla': 'TSLA'
}

# 컬럼명 바꾸기: Company → Ticker
financial_data['Ticker'] = financial_data['Company'].map(company_to_ticker)

# End Date를 Date로 복사해서 datetime 형식으로 변환
financial_data['Date'] = pd.to_datetime(financial_data['End Date'])



In [ ]:
# 필요한 재무 피처만 선택
features = ['Operating Income', 'Net Income', 'EPS Diluted', 'Total Assets', 'Shareholders Equity']

# 시작일 ~ 종료일 범위 설정 (주가 데이터 기준)
daily_index = pd.date_range(start='2020-01-01', end='2025-04-30', freq='D')

# 결과 저장용 리스트
daily_financial_list = []

# Ticker별로 반복
for ticker in financial_data['Ticker'].unique():
    df_t = financial_data[financial_data['Ticker'] == ticker].copy()
    df_t = df_t[['Date', 'Ticker'] + features]
    df_t.set_index('Date', inplace=True)
    df_t = df_t.sort_index()  # <-- 이 줄이 핵심
    df_t = df_t.reindex(daily_index, method='ffill')
    df_t['Ticker'] = ticker
    df_t = df_t.reset_index().rename(columns={'index': 'Date'})
    daily_financial_list.append(df_t)

# 모든 종목 합치기
daily_financial_data = pd.concat(daily_financial_list, ignore_index=True)

# 확인
print(len(daily_financial_data))
daily_financial_data.tail()

13629


,Date,Ticker,Operating Income,Net Income,EPS Diluted,Total Assets,Shareholders Equity
13624,2025-04-26,TSLA,399.0,409.0,0.12,125111.0,74653.0
13625,2025-04-27,TSLA,399.0,409.0,0.12,125111.0,74653.0
13626,2025-04-28,TSLA,399.0,409.0,0.12,125111.0,74653.0
13627,2025-04-29,TSLA,399.0,409.0,0.12,125111.0,74653.0
13628,2025-04-30,TSLA,399.0,409.0,0.12,125111.0,74653.0


In [ ]:
# 주가/기술 지표 + 기업 실적 데이터
mom_fin_merged_data = pd.merge(momentum_data, daily_financial_data, on=['Date', 'Ticker'], how='left')
print(len(mom_fin_merged_data))
mom_fin_merged_data.head()
data_B = mom_fin_merged_data.copy()

9135


In [ ]:
# 종가 + 기술지표 + 실적 + 거시지표 통합
full_merged = pd.merge(mom_fin_merged_data, Macro_data_daily, on='Date', how='left')

# 확인
print(full_merged.shape)
full_merged.head()
data_D = full_merged.copy()

(9135, 20)


# 2. Transformer 모델 훈련 분류(타겟: 다음 5일 수익률 UP/ Down)

## 2.1 전처리

### 2.1-1. 타겟 레이블 생성 (Up/Down 분류)

In [ ]:
def add_binary_target(df):
    df = df.copy()
    df['Target'] = (df['Close'].shift(-5) / df['Close']) - 1
    df['Target'] = (df['Target'] > 0).astype(int)
    return df

### 2.1-2 피처 정규화 (Z-score 방식 StandardScaler)

In [ ]:
from sklearn.preprocessing import StandardScaler

def apply_zscore_scaling(df):
    df = df.copy()
    non_feature_cols = ['Date', 'Ticker', 'Target']
    feature_cols = df.columns.difference(non_feature_cols)

    scaler = StandardScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols])
    return df


In [ ]:
# data_A: 종가 + 기술지표
# data_B: A + 실적
# data_C: A + 거시지표
# data_D: A + 실적 + 거시지표

data_A = apply_zscore_scaling(add_binary_target(data_A))
data_B = apply_zscore_scaling(add_binary_target(data_B))
data_C = apply_zscore_scaling(add_binary_target(data_C))
data_D = apply_zscore_scaling(add_binary_target(data_D))

### 2.1-3 시계열 윈도우 생성 함수

In [ ]:
import numpy as np

def create_sequence_data(df, window_size=30):
    df = df.dropna().reset_index(drop=True)
    feature_cols = df.columns.difference(['Date', 'Ticker', 'Target'])
    X, y = [], []
    values = df[feature_cols].values
    targets = df['Target'].values

    for i in range(len(df) - window_size):
        X.append(values[i:i+window_size])
        y.append(targets[i+window_size])  # 30일 뒤의 Target

    return np.array(X), np.array(y)


### 2.1-4 Transformer 모델 설계 (PyTorch)

In [ ]:
import torch
import torch.nn as nn

class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)  # feature → embedding
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 1),
            nn.Sigmoid()  # 이진 분류 확률
        )

    def forward(self, x):
        x = self.embedding(x)       # (B, T, d_model)
        x = self.transformer(x)     # (B, T, d_model)
        x = x[:, -1, :]             # 마지막 timestep 사용
        return self.classifier(x).squeeze(1)  # (B,)


### 2.1-5 모델 학습 함수 정의

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F

def train_transformer_model(X, y, epochs=10, batch_size=32, lr=0.001):
    # Tensor 변환
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    # 데이터셋 & 분할
    dataset = TensorDataset(X_tensor, y_tensor)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])

    # DataLoader
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    # 모델 초기화
    input_dim = X.shape[2]
    model = TimeSeriesTransformer(input_dim=input_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    # 학습 루프
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        # 검증
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                preds = model(xb)
                loss = loss_fn(preds, yb)
                val_losses.append(loss.item())

        print(f"Epoch {epoch+1:02d} | Train Loss: {np.mean(train_losses):.4f} | Val Loss: {np.mean(val_losses):.4f}")

    return model, val_loader


###2.1-6 예측 및 평가 (Accuracy / F1 Score)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

def evaluate_model(model, val_loader):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for xb, yb in val_loader:
            preds = model(xb)
            preds = (preds > 0.5).int()  # 확률 → 0 or 1
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(yb.int().cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)

    print(f"\n✅ Evaluation Results")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nDetailed Report:")
    print(classification_report(all_targets, all_preds))

    return acc, f1


### 2.1-7 4개 조합 실험 및 비교

In [ ]:
# 실험 대상 데이터셋 (이미 정규화 및 타겟 포함되어 있다고 가정)
experiments = {
    'A': data_A,
    'B': data_B,
    'C': data_C,
    'D': data_D
}

results = {}

for name, df in experiments.items():
    print(f"\n\n🚀 실험 {name} 시작: {len(df)} rows")

    # 1. 시계열 윈도우 생성
    X, y = create_sequence_data(df, window_size=30)

    # 2. 모델 학습
    model, val_loader = train_transformer_model(X, y, epochs=10)

    # 3. 성능 평가
    acc, f1 = evaluate_model(model, val_loader)

    # 4. 결과 저장
    results[name] = {'Accuracy': acc, 'F1 Score': f1}




🚀 실험 A 시작: 9135 rows
Epoch 01 | Train Loss: 0.6944 | Val Loss: 0.6883
Epoch 02 | Train Loss: 0.6889 | Val Loss: 0.6865
Epoch 03 | Train Loss: 0.6874 | Val Loss: 0.6854
Epoch 04 | Train Loss: 0.6853 | Val Loss: 0.6845
Epoch 05 | Train Loss: 0.6842 | Val Loss: 0.6841
Epoch 06 | Train Loss: 0.6853 | Val Loss: 0.6855
Epoch 07 | Train Loss: 0.6849 | Val Loss: 0.6848
Epoch 08 | Train Loss: 0.6840 | Val Loss: 0.6812
Epoch 09 | Train Loss: 0.6838 | Val Loss: 0.6828
Epoch 10 | Train Loss: 0.6837 | Val Loss: 0.6825

✅ Evaluation Results
Accuracy: 0.5700
F1 Score: 0.7028

Detailed Report:
              precision    recall  f1-score   support

           0       0.57      0.14      0.22       812
           1       0.57      0.92      0.70      1009

    accuracy                           0.57      1821
   macro avg       0.57      0.53      0.46      1821
weighted avg       0.57      0.57      0.49      1821



🚀 실험 B 시작: 9135 rows
Epoch 01 | Train Loss: 0.6933 | Val Loss: 0.6880
Epoch 02 | Tra

In [ ]:
import pandas as pd

# 표 형태로 결과 정리
result_df = pd.DataFrame(results).T  # .T: transpose for readability
result_df = result_df.round(4)
print("\n📈 실험별 성능 비교:")
display(result_df)



📈 실험별 성능 비교:


,Accuracy,F1 Score
A,0.5700,0.7028
B,0.5606,0.6873
C,0.6221,0.7099
D,0.5963,0.6570
